In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'


In [ ]:
import scipy
from scipy import signal
#import mxnet as mx

from matplotlib import pyplot as plt
from matplotlib import colors
#from mxnet import gluon,nd
import glob
import numpy as np
import os
from PIL import Image, ImageOps

# create dataloader (batch_size, 1, 100, 100)
def create_dataset(path, batch_size, shuffle):
  files = glob.glob(path)
  data = np.zeros((len(files),1,160,160))

  for idx, filename in enumerate(files):
    im = Image.open(filename)
    im = ImageOps.grayscale(im)
    im = im.resize((160,160))
    data[idx,0,:,:] = np.array(im, dtype=np.float32)/255.0

  #dataset = gluon.data.ArrayDataset(mx.nd.array(data, dtype=np.float32))
  #dataloader = gluon.data.DataLoader(dataset, batch_size=batch_size, last_batch="rollover", shuffle=shuffle)
  dataloader = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

  return dataloader

# create dataloader (batch_size, 10, 227, 227)
def create_dataset_stacked_images(path, batch_size, shuffle, augment=True):

  files = sorted(glob.glob(path))
  if augment:
    files = files + files[2:] + files[4:] + files[6:] + files[8:]
  data = np.zeros((int(len(files)/10),10,227,227))
  i, idx = 0, 0
  for filename in files:
    im = Image.open(filename)
    im = ImageOps.grayscale(im)
    im = im.resize((227,227))
    data[idx,i,:,:] = np.array(im, dtype=np.float32)/255.0
    i = i + 1
    if i > 9:
      i = 0
      idx = idx + 1
  #dataset = gluon.data.ArrayDataset(mx.nd.array(data, dtype=np.float32))
  #dataloader = gluon.data.DataLoader(dataset, batch_size=batch_size, last_batch="rollover", shuffle=shuffle)
  dataloader = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

  return dataloader

# perform inference and plot results
def plot_images(path, model, output_path="img", stacked=False, lstm=False):

  if not stacked:
    dataloader = create_dataset(path, batch_size=1, shuffle=False)
  else:
    dataloader = create_dataset_stacked_images(path, batch_size=1, shuffle=False, augment=False)

  counter = 0
  #model.load_parameters(params_file, ctx=ctx)

  try:
    os.mkdir(output_path,exist_ok=True)
  except:
    pass

  for image in dataloader:

    # perform inference
    #image = image.as_in_context(ctx)
    #if not lstm:
      #image_batch1=tf.transpose(image_batch,perm=[0,2,3,1])
      #reconstructed = model(image)
    #else:
      #states = model.temporal_encoder.begin_state(batch_size=1, ctx=ctx)
      #reconstructed, states = model(image, states)
    #compute difference between reconstructed image and input image
    #reconstructed = reconstructed.asnumpy()
    #image = image.asnumpy()
    #diff = np.abs(reconstructed-image)

    if not lstm:
      image_batch1=tf.transpose(image,perm=[0,2,3,1])
      reconstructed = model(image_batch1)
    else:
      states = model.temporal_encoder.begin_state(batch_size=1, ctx=ctx)
      reconstructed, states = model(image, states)
    #compute difference between reconstructed image and input image
    reconstructed = reconstructed.numpy()#tf.make_ndarray(reconstructed)
    image_batch1 = image_batch1.numpy()
    diff = np.abs(reconstructed-image_batch1)
    reconstructed=tf.transpose(reconstructed,perm=[0,3,1,2])
    diff=np.transpose(diff,(0,3,1,2))
    #image
    # in case of stacked frames, we need to compute the regularity score per pixel
    if stacked:
       image    = np.sum(image, axis=1, keepdims=True)
       reconstructed = np.sum(reconstructed, axis=1, keepdims=True)
       diff_max = np.max(diff, axis=1)
       diff_min = np.min(diff, axis=1)
       regularity = diff_max - diff_min
       # perform convolution on regularity matrix
       H = signal.convolve2d(regularity[0,:,:], np.ones((4,4)), mode='same')
    else:
      # perform convolution on diff matrix
      H = signal.convolve2d(diff[0,0,:,:], np.ones((4,4)), mode='same')

    # if neighboring pixels are anamolous, then mark the current pixel
    x,y = np.where(H > 4)

    # plt input image, reconstructed image and difference between both
    fig, (ax0, ax1, ax2,ax3) = plt.subplots(ncols=4, figsize=(10, 5))
    ax0.set_axis_off()
    ax1.set_axis_off()
    ax2.set_axis_off()

    ax0.imshow(image[0,0,:,:], cmap=plt.cm.gray, interpolation='nearest')
    ax0.set_title('input image')
    ax1.imshow(reconstructed[0,0,:,:], cmap=plt.cm.gray, interpolation='nearest')
    ax1.set_title('reconstructed image')
    ax2.imshow(diff[0,0,:,:], cmap=plt.cm.viridis, vmin=0, vmax=1, interpolation='nearest')
    ax2.set_title('diff ')
    ax3.imshow(image[0,0,:,:], cmap=plt.cm.gray, interpolation='nearest')
    ax3.scatter(y,x,color='red',s=0.3)
    ax3.set_title('anomalies')
    plt.axis('off')

     # save figure
    counter = counter + 1
    fig.savefig(output_path + "/" + str(counter) + '.png', bbox_inches = 'tight', pad_inches = 0.5)
    plt.close(fig)

In [ ]:
def hms_string(sec_elapsed):
  h = int(sec_elapsed / (60 * 60))
  m = int((sec_elapsed % (60 * 60)) / 60)
  s = sec_elapsed % 60
  return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [ ]:
train_directory='/*'
test_directory='/*'


In [ ]:
import glob
li=glob.glob(test_directory)

In [ ]:

output= plot_images(test_directory,model)